<a href="https://colab.research.google.com/github/edmyomet/Herb-BERT/blob/main/load_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
%pip install pdfplumber
%pip install pypdf2
%pip install pyPDF2
%pip install PyMuPDF

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.6/19.6 MB 65.7 MB/s eta 0:00:00


In [ ]:
import json
import pdfplumber

In [ ]:
from PyPDF2 import PdfReader
import fitz

# Extracting data from C.P. KHARE INDIAN MEDICINAL HERBS
### this data is extracted in order to train SLMs/LLMs to label images appropriately, and used to annotate research papers to further generate information on the herbs


In [ ]:
root_file_path:str=rf'/content/gdrive/MyDrive/herbology/herbology books/'
headings:list[str] = ['Synonym','Family','Habitat','English','Ayurvedic','Unani','Siddha/Tamil','Folk','Action','Dosage']

In [256]:
import fitz  # PyMuPDF

def herbology_detail_extraction(**kwargs) -> dict[str,dict[str,str]]:
    book = fitz.open(kwargs['file_path'])
    threshold_heading_size: float = 8.468199729919434
    threshold_title_size: float = 10.460800170898438
    title_print_style: int = 20
    data: dict[str, dict[str, str]] = {}

    for page_num in range(book.page_count):
        page = book.load_page(page_num)
        blocks = page.get_text("dict")['blocks']

        title: str = ""
        heading: str = ""
        current_text: str = ""

        for block in blocks:
            for line in block.get("lines", []):
                for span in line.get("spans", []):
                    text = span['text'].strip("\n").strip('▶')
                    font_size = span['size']
                    font_style = span['flags']
                    if font_size == threshold_title_size and font_style == title_print_style:
                        if title and heading:
                            data[title][heading] += current_text.strip()
                        title = text
                        heading = ""
                        current_text = ""
                        data[title] = {heading: "" for heading in headings}
                        continue


                    elif font_size == threshold_heading_size and text in headings:
                        if title:
                            if heading:
                                data[title][heading] += current_text.strip()
                            heading = text
                            current_text = ""
                            continue


                    if title and heading:
                        current_text += text + " "


        if title and heading:
            data[title][heading] += current_text.strip()
    #print(data)
    return data

In [257]:
def convert_to_json(**kwargs)->None:
    with open(kwargs['file_path'],'w+',encoding='utf-8') as json_file:
        json.dump(kwargs['data'],json_file, indent=4,ensure_ascii=False)

In [258]:
data = {"root":[herbology_detail_extraction(file_path=root_file_path+'book-4.pdf')]}
convert_to_json(file_path=root_file_path+f'json_data{1}.json', data=data)